# Part III: Feature Engineering

## Basic settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import warnings
from datetime import datetime, timedelta

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")
pd.options.display.max_rows = 50
pd.options.display.max_columns = None

In [3]:
src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [4]:
from utils.utils import save_data

In [5]:
DATA_DIR = "../data"

## Load preprocessed data

In [6]:
df_sales = pd.read_csv(
    os.path.join(DATA_DIR, "sales_data_preprocessed.csv"), parse_dates=["date"]
)
df_weather = pd.read_csv(
    os.path.join(DATA_DIR, "weather_preprocessed.csv"), parse_dates=["date"]
)

## Feature Engineering

In [7]:
# Start by creating a copy of the sales dataframe
df_features = df_sales.copy()

### Separating train and test

In [8]:
# Test set will be the last 3 months of 2017 (Oct, Nov, Dec 2017)
cutoff_date = pd.Timestamp("2017-10-01")
print(f"Train-test split date: {cutoff_date}")

# Create a flag for train/test split
df_features["is_test"] = df_features["date"] >= cutoff_date
print(f"Training df_features: {len(df_features[~df_features['is_test']])} rows")
print(f"Test df_features: {len(df_features[df_features['is_test']])} rows")

# Merge weather df_features
df_features = pd.merge(
    df_features,
    df_weather[["date", "province", "temperature", "humidity", "season"]],
    on=["date", "province"],
    how="left",
)

display(df_features)

Train-test split date: 2017-10-01 00:00:00
Training df_features: 175725 rows
Test df_features: 25300 rows


,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,season
0,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,1,Rice,10.0,False,13.4,87.8,winter
1,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,2,Noodles,13.0,False,13.4,87.8,winter
2,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,3,Bread,4.0,False,13.4,87.8,winter
3,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,4,Flour,6.0,False,13.4,87.8,winter
4,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,5,Cooking Oil,8.0,False,13.4,87.8,winter
...,...,...,...,...,...,...,...,...,...,...,...,...
201020,2017-12-31,Ho Chi Minh City,10,Binh Thanh Market,Baby & Health,31,Baby Wipes,15.0,True,25.9,67.7,dry
201021,2017-12-31,Ho Chi Minh City,10,Binh Thanh Market,Baby & Health,32,Pain Relievers,27.0,True,25.9,67.7,dry
201022,2017-12-31,Ho Chi Minh City,10,Binh Thanh Market,Baby & Health,33,Vitamins,14.0,True,25.9,67.7,dry
201023,2017-12-31,Ho Chi Minh City,10,Binh Thanh Market,Baby & Health,34,Cold & Flu Medicine,13.0,True,25.9,67.7,dry


### Add date and holidays related features

In [9]:
# Date features
df_features["year"] = df_features["date"].dt.year
df_features["month"] = df_features["date"].dt.month
df_features["day"] = df_features["date"].dt.day
df_features["day_of_week"] = df_features["date"].dt.dayofweek
df_features["is_weekend"] = df_features["day_of_week"].apply(
    lambda x: 1 if x >= 5 else 0
)
df_features["quarter"] = df_features["date"].dt.quarter

# For simplicity in this PoC, we'll consider some major holidays
# In a real scenario, you would have a more comprehensive list based on the specific market
holidays = [
    # Example Vietnamese holidays (simplification)
    "2016-01-01",
    "2016-01-02",
    "2016-01-03",  # New Year
    "2016-04-16",
    "2016-04-17",
    "2016-04-18",  # Hung Kings Festival
    "2016-04-30",
    "2016-05-01",
    "2016-05-02",  # Reunification Day and May Day
    "2016-09-02",
    "2016-09-03",
    "2016-09-04",  # National Day
    "2017-01-01",
    "2017-01-02",  # New Year
    # Add more holidays as needed
]
holidays = pd.to_datetime(holidays)
df_features["is_holiday"] = df_features["date"].isin(holidays).astype(int)

df_features.head()

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,season,year,month,day,day_of_week,is_weekend,quarter,is_holiday
0,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,1,Rice,10.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1
1,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,2,Noodles,13.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1
2,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,3,Bread,4.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1
3,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,4,Flour,6.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1
4,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,5,Cooking Oil,8.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1


### Weather-based features

In [10]:
# Temperature categories (using quantiles for even distribution)
temp_bins = [-np.inf, 20, 25, 30, np.inf]
temp_labels = ["Cold", "Cool", "Warm", "Hot"]
df_features["temp_category"] = pd.cut(
    df_features["temperature"], bins=temp_bins, labels=temp_labels
)

df_features.head()

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,season,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category
0,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,1,Rice,10.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold
1,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,2,Noodles,13.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold
2,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,3,Bread,4.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold
3,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,4,Flour,6.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold
4,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,5,Cooking Oil,8.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold


In [11]:
# Humidity levels
humidity_bins = [-np.inf, 60, 75, np.inf]
humidity_labels = ["Low", "Medium", "High"]
df_features["humidity_level"] = pd.cut(
    df_features["humidity"], bins=humidity_bins, labels=humidity_labels
)

df_features.head()

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,season,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category,humidity_level
0,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,1,Rice,10.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold,High
1,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,2,Noodles,13.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold,High
2,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,3,Bread,4.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold,High
3,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,4,Flour,6.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold,High
4,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,5,Cooking Oil,8.0,False,13.4,87.8,winter,2016,1,1,4,0,1,1,Cold,High


In [12]:
# Convert categorical variables to numeric
df_features = pd.get_dummies(
    df_features, columns=["temp_category", "humidity_level", "season"], drop_first=True
)

df_features.head()

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category_Cool,temp_category_Warm,temp_category_Hot,humidity_level_Medium,humidity_level_High,season_fall,season_spring,season_summer,season_wet,season_winter
0,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,1,Rice,10.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True
1,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,2,Noodles,13.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True
2,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,3,Bread,4.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True
3,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,4,Flour,6.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True
4,2016-01-01,Hanoi,1,Hoan Kiem Market,Staples,5,Cooking Oil,8.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True


### Last n days sales

In [13]:
# Sort df_features by date, store, and item for proper lag feature creation
df_features = df_features.sort_values(["date", "store_name", "item_name"])

# Create an identifier for store-item combinations for grouping
df_features["store_item"] = df_features["store_name"] + "_" + df_features["item_name"]

In [14]:
# Create lag features for sales - last n days
for n in [1, 7, 14, 28]:
    # Group by store_item and create lag features
    df_features[f"sales_lag_{n}"] = df_features.groupby("store_item")[
        "sales"
    ].transform(lambda x: x.shift(n))

### Rolling average features

In [15]:
# Create rolling average features
for window in [7, 14, 28]:
    # Mean of last n days
    df_features[f"sales_mean_{window}d"] = df_features.groupby("store_item")[
        "sales"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())

    # Min of last n days
    df_features[f"sales_min_{window}d"] = df_features.groupby("store_item")[
        "sales"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).min())

    # Max of last n days
    df_features[f"sales_max_{window}d"] = df_features.groupby("store_item")[
        "sales"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).max())

    # Standard deviation of last n days
    df_features[f"sales_std_{window}d"] = df_features.groupby("store_item")[
        "sales"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).std())

In [16]:
df_features.query("store_item == 'Ba Dinh Supermarket_Baby Wipes'")
# .to_csv("../data/check_data_01.csv", index=False)

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category_Cool,temp_category_Warm,temp_category_Hot,humidity_level_Medium,humidity_level_High,season_fall,season_spring,season_summer,season_wet,season_winter,store_item,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,sales_mean_7d,sales_min_7d,sales_max_7d,sales_std_7d,sales_mean_14d,sales_min_14d,sales_max_14d,sales_std_14d,sales_mean_28d,sales_min_28d,sales_max_28d,sales_std_28d
53,2016-01-01,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,18.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,2016-01-02,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,26.0,False,15.2,75.2,2016,1,2,5,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,18.0,NaN,NaN,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN
603,2016-01-03,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,14.0,False,19.2,86.0,2016,1,3,6,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,26.0,NaN,NaN,NaN,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854
878,2016-01-04,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,12.8,85.4,2016,1,4,0,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,14.0,NaN,NaN,NaN,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101
1153,2016-01-05,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,16.8,88.3,2016,1,5,1,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,24.0,NaN,NaN,NaN,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199703,2017-12-27,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,36.0,True,20.0,79.9,2017,12,27,2,0,4,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,34.0,55.0,34.0,25.0,45.285714,23.0,72.0,15.574705,40.285714,23.0,72.0,12.268964,36.357143,21.0,72.0,11.252043
199978,2017-12-28,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,41.0,True,17.2,67.2,2017,12,28,3,0,4,0,False,False,False,True,False,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,36.0,23.0,46.0,21.0,42.571429,23.0,72.0,15.251854,40.428571,23.0,72.0,12.201603,36.750000,21.0,72.0,11.030681
200253,2017-12-29,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,47.0,True,21.0,76.1,2017,12,29,4,0,4,0,True,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,41.0,42.0,30.0,27.0,45.142857,34.0,72.0,12.707328,40.071429,23.0,72.0,12.098724,37.464286,22.0,72.0,10.612648
200528,2017-12-30,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,58.0,True,20.9,80.8,2017,12,30,5,1,4,0,True,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,47.0,48.0,35.0,44.0,45.857143,34.0,72.0,12.641579,41.285714,23.0,72.0,11.860916,38.178571,22.0,72.0,10.555159


### Exponentially weighted moving average

In [17]:
# Exponentially weighted moving average
for alpha in [0.5, 0.75]:
    for window in [7, 14, 28]:
        # Create an EWMA feature
        alpha_str = str(alpha).replace(".", "")
        df_features[f"sales_ewma_{window}d_a{alpha_str}"] = df_features.groupby(
            "store_item"
        )["sales"].transform(lambda x: x.shift(1).ewm(alpha=alpha).mean())

In [18]:
df_features.query("store_item == 'Ba Dinh Supermarket_Baby Wipes'")

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category_Cool,temp_category_Warm,temp_category_Hot,humidity_level_Medium,humidity_level_High,season_fall,season_spring,season_summer,season_wet,season_winter,store_item,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,sales_mean_7d,sales_min_7d,sales_max_7d,sales_std_7d,sales_mean_14d,sales_min_14d,sales_max_14d,sales_std_14d,sales_mean_28d,sales_min_28d,sales_max_28d,sales_std_28d,sales_ewma_7d_a05,sales_ewma_14d_a05,sales_ewma_28d_a05,sales_ewma_7d_a075,sales_ewma_14d_a075,sales_ewma_28d_a075
53,2016-01-01,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,18.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,2016-01-02,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,26.0,False,15.2,75.2,2016,1,2,5,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,18.0,NaN,NaN,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
603,2016-01-03,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,14.0,False,19.2,86.0,2016,1,3,6,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,26.0,NaN,NaN,NaN,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854,23.333333,23.333333,23.333333,24.400000,24.400000,24.400000
878,2016-01-04,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,12.8,85.4,2016,1,4,0,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,14.0,NaN,NaN,NaN,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101,18.000000,18.000000,18.000000,16.476190,16.476190,16.476190
1153,2016-01-05,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,16.8,88.3,2016,1,5,1,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,24.0,NaN,NaN,NaN,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571,21.200000,21.200000,21.200000,22.141176,22.141176,22.141176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199703,2017-12-27,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,36.0,True,20.0,79.9,2017,12,27,2,0,4,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,34.0,55.0,34.0,25.0,45.285714,23.0,72.0,15.574705,40.285714,23.0,72.0,12.268964,36.357143,21.0,72.0,11.252043,42.116092,42.116092,42.116092,37.652005,37.652005,37.652005
199978,2017-12-28,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,41.0,True,17.2,67.2,2017,12,28,3,0,4,0,False,False,False,True,False,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,36.0,23.0,46.0,21.0,42.571429,23.0,72.0,15.251854,40.428571,23.0,72.0,12.201603,36.750000,21.0,72.0,11.030681,39.058046,39.058046,39.058046,36.413001,36.413001,36.413001
200253,2017-12-29,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,47.0,True,21.0,76.1,2017,12,29,4,0,4,0,True,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,41.0,42.0,30.0,27.0,45.142857,34.0,72.0,12.707328,40.071429,23.0,72.0,12.098724,37.464286,22.0,72.0,10.612648,40.029023,40.029023,40.029023,39.853250,39.853250,39.853250
200528,2017-12-30,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,58.0,True,20.9,80.8,2017,12,30,5,1,4,0,True,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,47.0,48.0,35.0,44.0,45.857143,34.0,72.0,12.641579,41.285714,23.0,72.0,11.860916,38.178571,22.0,72

### Store-level features

In [19]:
# Mean and sum of store sales in the last 7 days
df_features["store_mean_7d"] = df_features.groupby(["store_name", "date"])[
    "sales"
].transform("mean")
df_features["store_sum_7d"] = df_features.groupby(["store_name", "date"])[
    "sales"
].transform("sum")

### Item-level features

In [20]:
# Mean and sum of item sales in the last 7 days
df_features["item_mean_7d"] = df_features.groupby(["item_name", "date"])[
    "sales"
].transform("mean")
df_features["item_sum_7d"] = df_features.groupby(["item_name", "date"])[
    "sales"
].transform("sum")

In [21]:
df_features.query("store_item == 'Ba Dinh Supermarket_Baby Wipes'")

,date,province,store_id,store_name,category,item_id,item_name,sales,is_test,temperature,humidity,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category_Cool,temp_category_Warm,temp_category_Hot,humidity_level_Medium,humidity_level_High,season_fall,season_spring,season_summer,season_wet,season_winter,store_item,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,sales_mean_7d,sales_min_7d,sales_max_7d,sales_std_7d,sales_mean_14d,sales_min_14d,sales_max_14d,sales_std_14d,sales_mean_28d,sales_min_28d,sales_max_28d,sales_std_28d,sales_ewma_7d_a05,sales_ewma_14d_a05,sales_ewma_28d_a05,sales_ewma_7d_a075,sales_ewma_14d_a075,sales_ewma_28d_a075,store_mean_7d,store_sum_7d,item_mean_7d,item_sum_7d
53,2016-01-01,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,18.0,False,13.4,87.8,2016,1,1,4,0,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.785714,526.000000,21.000000,126.0
328,2016-01-02,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,26.0,False,15.2,75.2,2016,1,2,5,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,18.0,NaN,NaN,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN,18.000000,18.0,18.0,NaN,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,19.357143,542.000000,34.333333,206.0
603,2016-01-03,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,14.0,False,19.2,86.0,2016,1,3,6,1,1,1,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,26.0,NaN,NaN,NaN,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854,22.000000,18.0,26.0,5.656854,23.333333,23.333333,23.333333,24.400000,24.400000,24.400000,25.357143,710.000000,27.666667,166.0
878,2016-01-04,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,12.8,85.4,2016,1,4,0,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,14.0,NaN,NaN,NaN,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101,19.333333,14.0,26.0,6.110101,18.000000,18.000000,18.000000,16.476190,16.476190,16.476190,17.446891,488.512944,25.500000,153.0
1153,2016-01-05,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,24.0,False,16.8,88.3,2016,1,5,1,0,1,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,24.0,NaN,NaN,NaN,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571,20.500000,14.0,26.0,5.507571,21.200000,21.200000,21.200000,22.141176,22.141176,22.141176,17.785714,498.000000,25.833333,155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199703,2017-12-27,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,36.0,True,20.0,79.9,2017,12,27,2,0,4,0,False,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,34.0,55.0,34.0,25.0,45.285714,23.0,72.0,15.574705,40.285714,23.0,72.0,12.268964,36.357143,21.0,72.0,11.252043,42.116092,42.116092,42.116092,37.652005,37.652005,37.652005,31.196891,873.512944,58.666667,352.0
199978,2017-12-28,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,41.0,True,17.2,67.2,2017,12,28,3,0,4,0,False,False,False,True,False,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,36.0,23.0,46.0,21.0,42.571429,23.0,72.0,15.251854,40.428571,23.0,72.0,12.201603,36.750000,21.0,72.0,11.030681,39.058046,39.058046,39.058046,36.413001,36.413001,36.413001,30.571429,856.000000,62.000000,372.0
200253,2017-12-29,Hanoi,2,Ba Dinh Supermarket,Baby & Health,31,Baby Wipes,47.0,True,21.0,76.1,2017,12,29,4,0,4,0,True,False,False,False,True,False,False,False,False,True,Ba Dinh Supermarket_Baby Wipes,41.0,42.0,30.0,27.0,45.142857,34.0,72.0,12.707328,40.071429,23.0,72.0,12.098724,37.464286,22.0,72.0,10.612648,40.029023,40.029023,40.029023,3

In [22]:
# Drop rows with NaN (these will be the first n days without lag features)
df_features = df_features.dropna()

## Save feature engineered data

In [23]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193325 entries, 7752 to 200845
Data columns (total 55 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   date                   193325 non-null  datetime64[ns]
 1   province               193325 non-null  object        
 2   store_id               193325 non-null  int64         
 3   store_name             193325 non-null  object        
 4   category               193325 non-null  object        
 5   item_id                193325 non-null  int64         
 6   item_name              193325 non-null  object        
 7   sales                  193325 non-null  float64       
 8   is_test                193325 non-null  bool          
 9   temperature            193325 non-null  float64       
 10  humidity               193325 non-null  float64       
 11  year                   193325 non-null  int32         
 12  month                  193325 non-null  int32 

In [24]:
# PATH
num_features = df_features.shape[1]
save_path = os.path.join(DATA_DIR, f"feature_engineered_data_{num_features}_features.feather")
save_path

'../data/feature_engineered_data_55_features.feather'

In [25]:
# Save data
save_data(df_features, save_path, file_format='feather')

DataFrame saved to ../data/feature_engineered_data_55_features.feather in Feather format.
